In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)

In [3]:
df1 = pd.read_csv("tx.csv")
df1.shape

(21524, 102)

In [4]:
#df1.groupby('area_type')['area_type'].agg('count')

In [5]:
df2 = df1[['region_name','period_end','total_homes_sold','median_sale_price','median_sale_ppsf','total_active_listings','inventory_yoy']]
df2.shape
df2.isnull().sum()

region_name                 0
period_end                  0
total_homes_sold         2448
median_sale_price        2448
median_sale_ppsf         2459
total_active_listings       0
inventory_yoy             126
dtype: int64

In [6]:
#df2['average_homes_sold_with_price_drops_yoy'].fillna(value=0)

In [7]:
df3 = df2.dropna()
df3.shape

(19056, 7)

In [8]:
df3.head(10)

,region_name,period_end,total_homes_sold,median_sale_price,median_sale_ppsf,total_active_listings,inventory_yoy
1,"Crane County, TX",2022-08-21,3.0,133000.0,108.488264,183,0.400000
4,"Falls County, TX",2022-08-21,19.0,180457.1,96.386478,365,0.038462
5,"Falls County, TX",2022-08-21,7.0,112500.0,73.653394,110,-0.076923
6,"Hopkins County, TX",2022-08-21,10.0,254750.0,144.332102,103,0.476190
7,"Hopkins County, TX",2022-08-21,100.0,267812.5,151.656755,1132,0.349206
8,"Hopkins County, TX",2022-08-21,40.0,268812.5,150.023428,411,0.388060
9,"Williamson County, TX",2022-08-21,181.0,469000.0,242.624535,4240,0.958190
10,"Williamson County, TX",2022-08-21,2824.0,496035.1,248.478791,44262,0.915179
11,"Williamson County, TX",2022-08-21,871.0,490223.7,242.965810,16675,0.974735
12,"Bowie County, TX",2022-08-28,21.0,195000.0,104.074242,251,0.100478


In [9]:
df3.total_homes_sold.describe()

count    19056.000000
mean       290.513696
std       1149.664599
min          1.000000
25%          5.000000
50%         20.000000
75%         93.000000
max      21198.000000
Name: total_homes_sold, dtype: float64

In [10]:
df3.median_sale_price.describe()

count    1.905600e+04
mean     2.588998e+05
std      1.297152e+05
min      6.950000e+02
25%      1.850000e+05
50%      2.441645e+05
75%      3.153903e+05
max      3.800000e+06
Name: median_sale_price, dtype: float64

In [11]:
df3.total_active_listings.describe()

count     19056.000000
mean       4430.535422
std       17146.102475
min           1.000000
25%         130.000000
50%         470.000000
75%        1903.000000
max      313488.000000
Name: total_active_listings, dtype: float64

In [12]:
df3.inventory_yoy.describe()

count    19056.000000
mean         0.346367
std          0.622925
min         -0.833333
25%          0.000000
50%          0.218037
75%          0.565217
max         12.000000
Name: inventory_yoy, dtype: float64

In [13]:
def remove_total_homes_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('region_name'):
        m = np.mean(subdf.total_homes_sold)
        st = np.std(subdf.total_homes_sold)
        reduced_df = subdf[(subdf.total_homes_sold>(m-(st))) & (subdf.total_homes_sold<=(m+(st))) & (subdf.total_homes_sold<700)]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

df4 = remove_total_homes_outliers(df3)
df4.total_homes_sold.describe()

count    11851.000000
mean        57.766686
std        110.947262
min          1.000000
25%          4.000000
50%         14.000000
75%         51.000000
max        696.000000
Name: total_homes_sold, dtype: float64

In [14]:
def remove_median_sale(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('region_name'):
        m = np.mean(subdf.median_sale_price)
        st = np.std(subdf.median_sale_price)
        reduced_df = subdf[(subdf.median_sale_price>(m-(1.3*st))) & (subdf.median_sale_price<=(m+(1.3*st)))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

df5 = remove_median_sale(df4)
df4.median_sale_price.describe()

count    1.185100e+04
mean     2.516328e+05
std      1.214087e+05
min      2.300000e+04
25%      1.799000e+05
50%      2.373875e+05
75%      3.031549e+05
max      3.800000e+06
Name: median_sale_price, dtype: float64

In [15]:
def remove_total_listings(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('region_name'):
        m = np.mean(subdf.total_active_listings)
        st = np.std(subdf.total_active_listings)
        reduced_df = subdf[(subdf.total_active_listings>(m-(1.3*st))) & (subdf.total_active_listings<=(m+(1.3*st))) & (subdf.total_active_listings<5000)]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

df6 = remove_total_listings(df5)
df6.total_active_listings.describe()

count    7950.000000
mean      637.824528
std       949.249546
min         1.000000
25%        95.000000
50%       249.000000
75%       746.750000
max      4995.000000
Name: total_active_listings, dtype: float64

In [16]:
def remove_inventory(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('region_name'):
        m = np.mean(subdf.inventory_yoy)
        st = np.std(subdf.inventory_yoy)
        reduced_df = subdf[(subdf.inventory_yoy>(m-(1.3*st))) & (subdf.inventory_yoy<=(m+(1.3*st)))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

df7 = remove_inventory(df6)
df7.inventory_yoy.describe()

count    6530.000000
mean        0.250735
std         0.439018
min        -0.833333
25%        -0.009259
50%         0.180776
75%         0.444093
max         6.000000
Name: inventory_yoy, dtype: float64

In [17]:
def parse_date(df, features=[],drop_date=True):
    for feat in features:
        df[feat] = pd.to_datetime(df[feat])
        df[f'{feat}_year'] = df[feat].dt.year
        df[f'{feat}_month'] = df[feat].dt.month
        df[f'{feat}_day'] = df[feat].dt.day
        
        if drop_date: df.drop(columns=[feat],inplace=True)
        
    return df

In [18]:
df8 = df7.copy()
df8 = parse_date(df8,features=['period_end'])

In [19]:
df8.shape

(6530, 9)

In [20]:
df8.median_sale_ppsf.describe()

count    6530.000000
mean      139.298992
std        54.257253
min         9.488889
25%       106.563941
50%       135.245025
75%       166.684657
max       718.970897
Name: median_sale_ppsf, dtype: float64

In [21]:
def remove_ppsf_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('region_name'):
        m = np.mean(subdf.median_sale_ppsf)
        st = np.std(subdf.median_sale_ppsf)
        reduced_df = subdf[(subdf.median_sale_ppsf>(m-(1.5*st))) & (subdf.median_sale_ppsf<=(m+(1.5*st)))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

df9 = remove_ppsf_outliers(df8)
df9.median_sale_ppsf.describe()

count    5727.000000
mean      137.871835
std        49.773986
min        12.686156
25%       107.366086
50%       134.505015
75%       164.684559
max       651.953414
Name: median_sale_ppsf, dtype: float64

In [22]:
dummies = pd.get_dummies(df9.region_name)
dummies.head(3)

,"Abilene, TX metro area","Alice, TX metro area","Anderson County, TX","Andrews County, TX","Angelina County, TX","Aransas County, TX","Athens, TX metro area","Austin County, TX","Bandera County, TX","Beaumont, TX metro area",...,"Uvalde County, TX","Uvalde, TX metro area","Victoria, TX metro area","Walker County, TX","Waller County, TX","Ward County, TX","Washington County, TX","Williamson County, TX","Wise County, TX","Young County, TX"
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df10 = pd.concat([df9,dummies.drop('Young County, TX',axis='columns')],axis='columns')
df11 = df10.drop(['region_name','period_end_month','period_end_day'],axis='columns')

In [24]:
columns = df10.copy()
x = df11.drop(['total_homes_sold'],axis='columns')

In [25]:
y = df11.total_homes_sold
y.head()

0    140.0
1     31.0
2    125.0
3    189.0
4     90.0
Name: total_homes_sold, dtype: float64

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.23,random_state=42)

In [27]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(x_train.values,y_train)
lr_clf.score(x_test.values,y_test)

0.9237474674513222

In [28]:
def get_derived_parameters(region_name, df):
    region_data = df[df['region_name'] == region_name]
    
    median_sale_ppsf = region_data['median_sale_ppsf'].median()
    total_active_listings = region_data['total_active_listings'].median()
    inventory_yoy = region_data['inventory_yoy'].median()
    period_end_year = region_data['period_end_year'].median()
    
    return median_sale_ppsf, total_active_listings, inventory_yoy, int(period_end_year)

In [41]:
def predict_total_homes(region_name,median_price):
    try:
        loc_index = np.where(x.columns==region_name)[0][0]
    except:
        loc_index = -1
    
    region_data = df11[df10['region_name'] == region_name]
    
    median_sale_ppsf = region_data['median_sale_ppsf'].median()
    total_active_listings = region_data['total_active_listings'].median()
    inventory_yoy = region_data['inventory_yoy'].median()
    period_end_year = region_data['period_end_year'].median()
    
    X = np.zeros(len(x.columns))
    X[0] = median_price
    X[1] = median_sale_ppsf
    X[2] = total_active_listings
    X[3] = inventory_yoy
    X[4] = period_end_year
    if loc_index >= 0:
        X[loc_index] = 1
        
    return (round(lr_clf.predict([X])[0],0))
    

In [43]:
#Here is where we can input values and get an actual prediction
#The first parameter is the county that we want to estimate from
#The second parameter is the median sale price of homes that will be sold

predict_total_homes("Crosby County, TX", 100000)

5.0